In [ ]:
import pandas as pd
import SALib
from pathlib import Path
import os
import subprocess
import sys
import matplotlib.pyplot as plt
from IPython import display

To view this notebook as slides, run `jupyter nbconvert Tutorial.ipynb --to slides --post serve` on the command line

# ECEMP Skills Workshop: Global Sensitivity Analysis

- Will Usher, KTH Royal Institute of Technology
- Trevor Barnes, Simon Fraser University

Join the [menti](https://www.menti.com/alhebjoi4h3i) by going to [menti.com](https://www.menti.com) and using the code 4649 2930 (available between 5-7th October).

# Introduction

## What is global sensitivity analysis?

- Global sensitivity analysis is “*the study of how uncertainty in the output of a model...can be apportioned to different sources of uncertainty in the model input*”. 
- “Global” means that all parameters are moved over their input ranges at the same time.
- By contrast, in one-at-a-time (OAT) sensitivity analysis, only one dimension of the model input space is explored while holding all other dimensions at their central value.
- An excellent introduction to global sensitivity analysis can be found in Saltelli (2007).

## Why perform global sensitivity analysis?

- Most energy system optimisation modelling studies use scenario analysis
- Global sensitivity analysis can supplement and support scenario analysis
  - identify key drivers of model results (factor prioritisation)
  - identify unimportant parameters (factor fixing)
  - identify interesting areas of the input space (factor mapping)

## Methods for conducting global sensitivity analysis

Variance-based approaches compute the proportion of the variance in the output that is explained by variation in the input. 

- Computation of a first-order (only the direct effect) and total-order (direct and interaction effects) index for each input recovers all interaction and non-linear effects of a model. 
- Variance-based approaches require a relatively large number $(500(k+2))$ of model runs.

Elementary effects test, or “Method of Morris” lowers computationally needs $(10(k+1))$ by computing an average of local derivatives over a discrete space of input parameters.

- It provides an estimate for the total-order index produced by more advanced variance-based approaches 
- can be applied to groups of inputs to increase coverage and reduce computational demands.

## Tools for performing global sensitivity analyses

The Python library [SALib](http://salib.github.io/SALib/) provides implementations of powerful global sensitivity analysis approaches.


-   Sobol Sensitivity Analysis ([Sobol
    2001](http://www.sciencedirect.com/science/article/pii/S0378475400002706), [Saltelli
    2002](http://www.sciencedirect.com/science/article/pii/S0010465502002801), [Saltelli et al.
    2010](http://www.sciencedirect.com/science/article/pii/S0010465509003087))

-   Method of Morris, including groups and optimal trajectories ([Morris
    1991](http://www.tandfonline.com/doi/abs/10.1080/00401706.1991.10484804), [Campolongo et al.
    2007](http://www.sciencedirect.com/science/article/pii/S1364815206002805))

-   Fourier Amplitude Sensitivity Test (FAST) ([Cukier et al.
    1973](http://scitation.aip.org/content/aip/journal/jcp/59/8/10.1063/1.1680571), [Saltelli et al.
    1999](http://amstat.tandfonline.com/doi/abs/10.1080/00401706.1999.10485594))

-   Random Balance Designs - Fourier Amplitude Sensitivity Test
    (RBD-FAST) ([Tarantola et al.
    2006](https://hal.archives-ouvertes.fr/hal-01065897/file/Tarantola06RESS_HAL.pdf), [Elmar Plischke
    2010](https://doi.org/10.1016/j.ress.2009.11.005), [Tissot et al.
    2012](https://doi.org/10.1016/j.ress.2012.06.010))

-   Delta Moment-Independent Measure ([Borgonovo
    2007](http://www.sciencedirect.com/science/article/pii/S0951832006000883), [Plischke et al.
    2013](http://www.sciencedirect.com/science/article/pii/S0377221712008995))

-   Derivative-based Global Sensitivity Measure (DGSM) ([Sobol and
    Kucherenko
    2009](http://www.sciencedirect.com/science/article/pii/S0378475409000354))

-   Fractional Factorial Sensitivity Analysis ([Saltelli et al.
    2008](http://www.wiley.com/WileyCDA/WileyTitle/productCd-0470059974.html))

-   High Dimensional Model Representation ([Li et al.
    2010](https://pubs.acs.org/doi/pdf/10.1021/jp9096919))
    
   

In [ ]:
from SALib import ProblemSpec
from SALib.test_functions import Ishigami

To use the SALib libary, you first define a `ProblemSpec` containing the names and bounds of your variables.

In [ ]:
# By convention, we assign to "sp" (for "SALib Problem")
sp = ProblemSpec({
  'names': ['x1', 'x2', 'x3'],    # Name of each parameter
  'bounds': [[-np.pi, np.pi]]*3,  # bounds of each parameter
  'outputs': ['Y']                # name of outputs in expected order
})

Then you create a sample using the method of your choice.

In [ ]:
# Create the sample
sp = sp.sample_saltelli(1024, calc_second_order=True)

You run your model many times, once for each row of the sample.

In [ ]:
# Run the model
sp = sp.evaluate(Ishigami.evaluate)

Finally, you run the matching analysis method for each model output you wish to perform the global sensitivity analysis.

In [ ]:
# Run the analysis method
sp = sp.analyze_sobol(print_to_console=False)

In [ ]:
# Samples, model results and analyses can be extracted from the ProblemSpec
print(sp)

In [ ]:
# Basic plotting functionality is also provided
sp.plot()

In [ ]:
from SALib import ProblemSpec
from SALib.test_functions import Ishigami

# By convention, we assign to "sp" (for "SALib Problem")
sp = ProblemSpec({
  'names': ['x1', 'x2', 'x3'],    # Name of each parameter
  'bounds': [[-np.pi, np.pi]]*3,  # bounds of each parameter
  'outputs': ['Y']                # name of outputs in expected order
})

# Create the sample
sp.sample_morris(100, optimal_trajectories=10, local_optimization=True, num_levels=4)

# Run the model
sp.evaluate(Ishigami.evaluate)

# Run the matching analysis method
sp.analyze_morris(print_to_console=False)

# Samples, model results and analyses can be extracted:
print(sp)

# Basic plotting functionality is also provided
sp.plot()

# References

Andrea Saltelli, Marco Ratto, Terry Andres, Francesca Campolongo, Jessica Cariboni, Debora Gatelli, Michaela Saisana, and Stefano Tarantola. Global Sensitivity Analysis. The Primer. 1st ed. John Wiley & Sons, Ltd, 2007. [doi:10.1002/9780470725184](https://doi.org/10.1002/9780470725184).

Iwanaga, T., Usher, W., & Herman, J. (2022). Toward SALib 2.0: Advancing the accessibility and interpretability of global sensitivity analyses. Socio-Environmental Systems Modelling, 4, 18155. [doi:10.18174/sesmo.18155](https://doi.org/10.21105/joss.00097)

Herman, J. and Usher, W. (2017) SALib: An open-source Python library for sensitivity analysis. Journal of Open Source Software, 2(9). [doi:10.21105/joss.00097](https://doi.org/10.21105/joss.00097)

# Example One

In this example, we will first show build a simple model and perform 
**LOCAL** sensitivity analysis

## Reference Energy System

Consider the simple energy system optimization model below. It includes: 

- A mining technology to introduce uranium 
- A nuclear power generation technology
- Uranium fuel
- Electricity fuel

![RES](./examples/Example_One/res.jpg)

## Input Data

We will add data for: 
- Capital Costs
- Fixed Costs
- Variable Costs (including fuel costs)
- Powerplant Efficiency 
- Powerplant Operational Life 
- Electricity Demand 

The model will be run given the assumptions: 
- Model horizon of 50 years 
- 8 equal time slices per year
- Linearly increasing yearly demand

In [ ]:
# REFERENCE ONLY. 
# DATA ALREADY CREATED. 
# DO NOT CHANGE
data_dir = Path('./examples/Example_One/')

YEARS = range(2020, 2070)
TECHNOLOGIES = ['MINE_URANIUM', 'NUCLEAR']
FUELS = ['URANIUM', 'ELECTRICITY']
REGIONS = ['R1']
TIME_SLICE = ['S1D1', 'S1D2', 'S2D1', 'S2D2','S3D1','S3D2','S4D1','S4D2']
YEAR_SPLIT = 0.125
MODES = [1]

### Add Capital Costs

In [ ]:
def add_capex(cost):
    years = range(2020, 2070)
    df = pd.DataFrame(
        [['R1', 'NUCLEAR', years[0], cost]] * len(years), 
        columns=['REGION', 'TECHNOLOGY', 'YEAR', 'VALUE'])
    df['YEAR'] = years
    df.to_csv(Path(data_dir, 'data', 'CapitalCost.csv'), index=False)
    return df

In [ ]:
capex = 4000 # M$ / GW
add_capex(capex).head()

### Add Fixed Operational Costs

In [ ]:
def add_fixed(cost):
    years = range(2020, 2070)
    df = pd.DataFrame(
        [['R1', 'NUCLEAR', YEARS[0], cost]] * len(YEARS), 
        columns=['REGION', 'TECHNOLOGY', 'YEAR', 'VALUE'])
    df['YEAR'] = YEARS
    df.to_csv(Path(data_dir, 'data', 'FixedCost.csv'), index=False)
    return df

In [ ]:
fixed = 50 # M$ / GW
add_fixed(fixed).head()

### Add Variable Costs

In [ ]:
def add_variable(cost):
    years = range(2020, 2070)
    df = pd.DataFrame(
        [['R1', 'MINE_URANIUM', 1, years[0], cost]] * len(YEARS), 
        columns=['REGION','TECHNOLOGY','MODE_OF_OPERATION','YEAR','VALUE'])
    df['YEAR'] = YEARS
    df.to_csv(Path(data_dir, 'data', 'VariableCost.csv'), index=False)
    return df

In [ ]:
variable = 2.5 # M$ / PJ
add_variable(variable).head()

### Add Nuclear Power Plant Efficiency 

In [ ]:
def add_efficiency(eff):
    years = range(2020, 2070)
    df = pd.DataFrame(
        [['R1', 'NUCLEAR', 'URANIUM', 1, years[0], eff]] * len(years), 
        columns=['REGION','TECHNOLOGY','FUEL','MODE_OF_OPERATION','YEAR','VALUE'])
    df['YEAR'] = years
    df.to_csv(Path(data_dir, 'data', 'InputActivityRatio.csv'), index=False)
    return df

In [ ]:
eff = 1.25 # 80 %
add_efficiency(eff).head()

### Add Demand

In [ ]:
def add_demand(start_value, yearly_increase):
    demand_data = []
    for year in range(2020, 2070):
        demand_data.append([
            'R1', 
            'ELECTRICITY',
            year,
            start_value*(1+yearly_increase)**(year-YEARS[0])
        ])
    df = pd.DataFrame(demand_data, columns=['REGION','FUEL','YEAR','VALUE'])
    df.to_csv(Path(data_dir, 'data', 'SpecifiedAnnualDemand.csv'), index=False)
    return df

In [ ]:
start_value = 1000 # PJ
yearly_increase = 0.05 # %
add_demand(start_value, yearly_increase).head()

### Add Operational Life

In [ ]:
def add_operational_life(op_life):
    df = pd.DataFrame([['R1','NUCLEAR',op_life]], columns=['REGION','TECHNOLOGY','VALUE'])
    df.to_csv(Path(data_dir, 'data', 'OperationalLife.csv'), index=False)
    return df

In [ ]:
op_life = 50 # years
add_operational_life(op_life).head()

## One-at-a-time Sensitivity Analysis

Run the model with the baseline input parameters and get a reference objective value

In [ ]:
import subprocess

def run_model(path_to_data_dir, path_to_model_file, result_file_name):
    
    subprocess.check_output(
        [
            "otoole", "convert", "datapackage", "datafile", 
            Path(path_to_data_dir,'datapackage.json'), Path(path_to_data_dir,'data.txt')
        ])

    subprocess.check_output(
        [
            "glpsol", 
            "-m", Path(path_to_model_file), 
            "-d", Path(path_to_data_dir,'data.txt'), 
            "--wlp", Path(path_to_data_dir,'model.lp'), "--check"
        ])
    subprocess.check_output(
        [
            "cbc", Path(path_to_data_dir,'model.lp'), "solve", 
            "-solu", Path(path_to_data_dir,result_file_name)
        ])
#     os.system(
#         'cbc {} solve -solu {}' .format(
#             Path(path_to_data_dir,'model.lp'),
#             Path(path_to_data_dir,result_file_name)
#         )
#     )

def get_objective_value(result_files):
    for result_file in result_files:
        print(f'{result_file.name}')
        os.system('head -1 {}' .format(Path(result_file)))

In [ ]:
run_model(data_dir, './examples/Example_One/osemosys.txt', 'solution.sol')
get_objective_value([Path(data_dir, 'solution.sol')])

## Model Results

Solving the model for total discounted costs produces an objective value  of `$572,721,000,000`. In local sensitivity analysis, we select a single parameter to alter and rerun the model. This will tell us the effect that single parameter has on the results around the optimal location. 

### Parameter Changes

We will change both the **capital costs** and **variable costs** of the nuclear power plant to see how sensitive the results are to these parameters. We will do 5 additional runs for each parameter, resulting in 10 total runs. This summary of model runs and results are given below.  

 Run      | Capital Costs (M\$/GW) | Variable Costs (M\$/PJ) | Objective Cost (M$)
:---|---:|---:|---:
 Original | 4000                  | 2.5                    | 572721              
 1        | 2500                  | 2.5                    | 446506              
 2        | 3000                  | 2.5                    | 488577              
 3        | 3500                  | 2.5                    | 530649              
 4        | 4500                  | 2.5                    | 614793              
 5        | 5000                  | 2.5                    | 656865              
 6        | 4000                  | 1.5                    | 511727              
 7        | 4000                  | 2.0                    | 542224              
 8        | 4000                  | 3.0                    | 572721              
 9        | 4000                  | 3.5                    | 633715              
 10       | 4000                  | 4.0                    | 664212              

In [ ]:
result_files = []

# Iterate over capital cost
capital_costs = [2500, 3000, 3500, 4500, 5000]
for capital_cost in capital_costs:
    add_capex(capital_cost)
    result_file = f'capex_{capital_cost}.txt'
    result_files.append(Path(data_dir, result_file))
    run_model(data_dir, './examples/Example_One/osemosys.txt', result_file)

# resest to origianl capital cost
add_capex(4000)

# Iterate over variable cost
var_costs = [1.5, 2.0, 3.0, 3.5, 4.0]
for var_cost in var_costs:
    add_variable(var_cost)
    result_file = f'var_cost_{var_cost}.txt'
    result_files.append(Path(data_dir, result_file))
    run_model(data_dir, './examples/Example_One/osemosys.txt', result_file)

# reset to original variable cost
add_variable(2.5)

# extract objective costs
get_objective_value(result_files)

Create simple plots to see how the costs change with parameters

In [ ]:
def plot_LSA(df_capex, df_var):
    fig, axs = plt.subplots(1,2, figsize=(14,5), sharey=True)
    df_capex.plot(ax=axs[0], marker='o', title='Effects of Capital Cost', xlabel='Capital Cost (M$/GW)', ylabel='Objective Cost (M$)')
    df_var.plot(ax=axs[1], marker='o', title='Effects of Variable Cost', xlabel='Variable Cost (M$/PJ)', ylabel='Objective Cost (M$)')

In [ ]:
df_capex = pd.DataFrame([
    [2500, 446506],
    [3000, 488577],
    [3500, 530649],
    [4000, 572721],
    [4500, 614793],
    [5000, 656865],
], columns=['Capex', 'Objective_Cost']).set_index('Capex')

df_var = pd.DataFrame([
    [1.5, 511727],
    [2.0, 542224],
    [2.5, 572721],
    [3.0, 603218],
    [3.5, 633715],
    [4.0, 664212],
], columns=['Var_Cost', 'Objective_Cost']).set_index('Var_Cost')

In [ ]:
plot_LSA(df_capex, df_var)

## Disadvantages of local sensitivity analysis

A local sensitivity analysis has several weaknesses.  These include the following.

### Limited insights into the influence of parameters

The steeper gradiant of the capital cost plot shows this is the more influential parameter over our range of values. As we add more parameters and build more complex models, viewing the gradient will become impractical. 

### Missing interaction effects

This example also shows a relatively linear responce between our variables and value of the objective function. While non-linear effects may be identified by this method, interaction effects would not be. As discussed later, variable costs (which include fuel cost) and the efficiency of the power plant will interact. We do not see this interaction through local sensitivity analysis.

### Computational expense

This approach is inefficient as we need to run one model run per parameter per step.

# Example Two 

We will repeat example one, except employ **GLOBAL** sensitivity analysis using the Method of Morris.

## Reference Energy Systen

Our RES and included parameters will remain the same as example 1. 

![RES](./examples/Example_Two/res.jpg)

## The Workflow

Performing global sensitivity analysis requires running the model many times and careful handling of the results. Therefore, it is beneficial to set up an automated workflow to reduce data handling errors. To execute the workflow, three main steps are required. 

1. Modify the configuration file
2. Run the workflow 
3. Interpret the results

A high-level graphical overview of the the workflow is given below. 


![workflow](./examples/Example_Two/workflow.jpg)


## The Configuration File

The configuration file contains parameters that define how the sensitivity analysis will be conducted. For each factor we want to test, we can test its sensitivity independently of time. Moreover, if the parameter is defined over time (such as a yearly changing demand), we can also test the sensitivity dependent on time. 

### Time Independent Factors 

The figure below highlights the two different options for time-independent factors that can be configured in the workflow. The red lines prepresent the user-entered bounds, while the green lines represent the modelled values. 

![gsa-options](./examples/Example_Two/gsa_options_fixed.jpg)


### Option One

We specify a fixed start point. This results in **no sensitivity analysis** being performed, because we are not varying the parameter over different model runs. 

### Option Two

We specify a moving start point, represented by the red bars. This can be beneficial in situations where we know the parameter value will not change throughout the model run, but want to test the sensitivity of this fixed value. For example, the operational life of a powerplant. 

### Time Dependent Factors 

The figure below highlights three different options for time-dependent factors that can be configured in the workflow. The red lines represent the user-entered bounds, while the green lines represent the interpolated values. 

![gsa-options](./examples/Example_Two/gsa_options_interpolate.jpg)


### Option One

We specify fixed start and end points. This results in **no sensitivity analysis** being performed, because we are not varying the parameter over different model runs. 

### Option Two

We specify a fixed start point, and a moving end point. This can be beneficial in scenario where we know the starting value, but are uncertain about how the value will change in time. For example, the annual demand of our start year may be a known value, but we are uncertain how this will progress throughout the model horizon.
Note that we currently interpolate linearly between the two points but this could include different shapes in the future.

### Option Three

We specify moving start and end points. This is appropriate to test the sensitivity of values when uncertain of both current and future values. For example, fuel costs are always changing, and testing the sensitivity of these changes on your model may be important.

### Modify the Configuration File

Navigate to [`examples/Example_Two/parameters.csv`](../edit/examples/Example_Two/parameters.csv) file and add sensitivity parameters for the `CapitalCost`, `VariableCost`, and `InputActivityRatio` (defined as 1/efficiency). The formatted parameter csv should look similar to the following. The `interpolation_index` value is either `fixed` or `interpolate` for time independent and time dependent parameters respectively. 

Feel free to user your own values for the min/max values!

|name |group|indexes|min_value_base_year|max_value_base_year|min_value_end_year|max_value_end_year|dist|interpolation_index|action|
|-----|-----|-------|-------------------|-------------------|------------------|------------------|----|-------------------|------|
|CapitalCost  |capital  |"R1,NUCLEAR"        |4000 |5000 |2000 |3000 |"unif" |"YEAR" |"interpolate" |
|VariableCost |variable |"R1,MINE_URANIUM,1" |3.5  |4.0  |1.5  |1.75 |"unif" |"YEAR" |"interpolate" |
|InputActivityRatio | IAR |"R1,NUCLEAR,URANIUM,1" |1.3 | 1.5 | 1.05 | 1.2 | "unif" | "YEAR" | "interpolate" |

### Run the Workflow

Once we have defined our sensitivity parameters, we can run the workflow

In [ ]:
!snakemake --cores 4  clean --configfile examples/Example_Two/config.yaml --quiet

In [ ]:
!snakemake --cores 4 --configfile examples/Example_Two/config.yaml --quiet

## View Results 

The global sensitivity analysis method used is Method of Morris (or elementary effects). 
This screening method produces three sensitivity measures, $mu$, $mu*$, and $\sigma$.

- $mu$ estimates the mean of the distribution of the elementary effects
- $mu*$ estimates the mean of the distribution of the absolute elementary effects 
- $\sigma$ (the standard deviation) provides an indication of the nonlinear effects and/or interactions with other factors

The purpose of calculating $mu*$ in addition to $mu$ is that $mu$ may fail to identify a factor with considerable influence on the model. This can happen when both positive and negative effects are present, which are then cancelled out.

### Tabular Results

Tabluar results are found in the file [`results/SA_0.csv`](../edit/results/SA_0.csv)

In [ ]:
pd.read_csv('results/SA_0.csv', index_col=0)

### Graphical Results

Graphical results are found in the file [`results/0_summary/SA_objective.png`](results/0_summary/SA_objective.png)

In [ ]:
display.Image('./results/SA_0.png')

### Interpreting Results

Capital cost is the most influential factor, of the ones tested, on the total discounted cost of the system. There is no interaction between capital costs and variable costs or input activity ratio. Variable costs and the input activity ratio have non-linear effects and/or interact with each other. 

# Example 3

In this example, it's now up to you to modify the sensitivity parameters ! 

## Reference Energy System

Our RES and included parameters will remain the same. 

![RES](./examples/Example_Three/res.jpg)

### Modify the Configuration File

Navigate to [`examples/Example_Three/parameters.csv`](../edit/examples/Example_Three/parameters.csv) file and add sensitivity parameters. The available parameters to explore are shown below. Modify the configuration file values however you want ! 

|name |group|indexes|min_value_base_year|max_value_base_year|min_value_end_year|max_value_end_year|dist|interpolation_index|action|
|-----|-----|-------|-------------------|-------------------|------------------|------------------|----|-------------------|------|
|CapitalCost           |capital  |"R1,NUCLEAR"                |XXX |XXX |XXX |XXX |"unif" |"YEAR" |"interpolate" |
|FixedCost             |fixed    |"R1,NUCLEAR"                |XXX |XXX |XXX |XXX |"unif" |"YEAR" |"interpolate" |
|VariableCost          |variable |"R1,MINE_URANIUM,1"         |XXX |XXX |XXX |XXX |"unif" |"YEAR" |"interpolate" |
|SpecifiedAnnualDemand |demand   |"R1,ELECTRICITY"            |XXX |XXX |XXX |XXX |"unif" |"YEAR" |"interpolate" |
|InputActivityRatio    |iar      |"R1,NUCLEAR,URANIUM,1"      |XXX |XXX |XXX |XXX |"unif" |"YEAR" |"interpolate" |
|OperationalLife       |oplife   |"R1,NUCLEAR"                |XXX |XXX |XXX |XXX |"unif" |"YEAR" |"interpolate" |

### Run the Workflow

In [ ]:
!snakemake clean --cores 4 --configfile examples/Example_Three/config.yaml --quiet

In [ ]:
!snakemake --cores 4 --configfile examples/Example_Three/config.yaml --quiet

### View Results

In [ ]:
pd.read_csv('results/SA_0.csv', index_col=0)

In [ ]:
display.Image('./results/SA_0.png')

# Summary 

In this tutorial we have explored how global sensitivity analysis can be applied to very simple energy system optimization model. We applied the Method of Morris to identify influential parameters and parameters that have non-linear and/or interaction with other parameters. In comparison to local sensitivity analysis, global sensitivity analysis allows us explore a wide array of the solution space and identify the important factors. 

During this tutorial, we looked at the sensitivity of one out, total discounted system cost, but we can perform sensitivity analysis for any model output variable of interest. For example, installed capacity, emissions or production. 

We used the Method of Morris to conduct the sensitivity analysis as it allows us to extract sufficient preliminary information with minimal model runs. This is beneficial as energy system optimization models can often have multi-hour run times. Other variance based global sensitivity methods, including those provided by the Python library SALib, can provide more information into the behaviour of the model, with the caveat that this often requires more model runs.